<a href="https://colab.research.google.com/github/thinhngo-x/Anonymization/blob/master/notebook/Anonymization_XLNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Anonymization/CONLL2003_XLNET/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
! pip install transformers --quiet

In [0]:
import transformers
import os
from torch.nn import CrossEntropyLoss
import logging
logger = logging.getLogger(__name__)

In [0]:
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testa" --quiet
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.train" --quiet
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testb" --quiet

### Some useful functions

In [0]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode="eng.testa"):
    file_path = os.path.join(data_dir, "{}".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features

### Obtaining the representation of the dataset from XLNet

#### An example

In [0]:
from transformers import XLNetTokenizer, XLNetModel
import torch

In [0]:
model = XLNetModel.from_pretrained("xlnet-large-cased").cuda()
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")

sequence = "Adrien Ehrhardt donne un projet d'informatique aux étudiants de INF442."
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs.cuda())[0]

In [0]:
outputs.detach().cpu().numpy()[0].shape

In [0]:
outputs

In [0]:
del outputs

#### The real thing

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

In [0]:
from tqdm import tqdm, trange

In [0]:
import numpy as np

In [0]:
pad_token_label_id=CrossEntropyLoss().ignore_index
labels = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
model = XLNetModel.from_pretrained("xlnet-large-cased").cuda()
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")
examples = read_examples_from_file(".", mode="eng.testa")
features = convert_examples_to_features(
            examples,
            label_list = labels,
            max_seq_length = 128,
            tokenizer = tokenizer,
            cls_token_at_end=True,
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=2,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,
            pad_on_left=False,
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
            pad_token_label_id=pad_token_label_id)

In [0]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

In [0]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=32)
preds = None
out_label_ids = None

In [14]:
model.eval()
preds = []
out_label_ids = []
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    batch = tuple(t.to("cuda") for t in batch)

    with torch.no_grad():
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        inputs["token_type_ids"] = (
            batch[2]
        )
        outputs = model(**inputs)
        last_hidden_layer = outputs[0]
        preds.append(last_hidden_layer.detach().cpu().numpy())
        out_label_ids.append(batch[3].detach().cpu().numpy())

Evaluating: 100%|██████████| 102/102 [02:00<00:00,  1.18s/it]


In [0]:
out_label_list = [[] for _ in range(len(out_label_ids) * 32)]
preds_list = [[] for _ in range(len(out_label_ids) * 32)]

label_map = {i: label for i, label in enumerate(labels)}
        
for i in range(len(out_label_ids)):
    for b in range(out_label_ids[i].shape[0]):
        for j in range(out_label_ids[0].shape[1]):
            if out_label_ids[i][b,j] != pad_token_label_id:
                out_label_list[i*32+b].append(out_label_ids[i][b,j])
                preds_list[i*32+b].append(preds[i][b,j])

`out_label_list` is of shape (#number of sentences, #tokens in this sentence, 1)

`preds_list` is of shape (#number of sentences, #tokens in this sentence, 1024 hidden representations)

For the sake of simplicity, we'll consider that an observation / a sample is a token, not a sentence (the contextual meaning of each token is already taken into account in its representation), so we need to "flatten" both lists so that they're of shape (#tokens, 1)

In [0]:
import itertools

In [0]:
true_labels = np.array(list(itertools.chain.from_iterable(out_label_list)))

In [0]:
flat_list = []
for sentence in preds_list:
    for token in sentence:
        flat_list.append(token)
        
representation = np.array(flat_list)

In [19]:
true_labels.shape

(51236,)

In [20]:
representation.shape

(51236, 1024)

In [0]:
np.save(root+"true_labels.train.npy", true_labels)

In [0]:
np.save(root+"representation.train.npy", representation)